In [10]:
from sklearn.model_selection import KFold
def cross_validation(folds, data, labels, error_function, model, **model_args):
    kf = KFold(n_splits=folds, random_state=None, shuffle = True)
    score = []
    y = labels
    if model == LogisticRegression:
        M = model(max_iter = 20**9)
    else:
        M = model()
    predicted_lable = pd.DataFrame(columns = ['prediction'])
    for train_index,test_index in kf.split(data):
        x_training_set = data.iloc[train_index]
        y_training_set = y.iloc[train_index]
        x_test_set = data.iloc[test_index]
        y_test_set = y.iloc[test_index]
        M.fit(np.array(x_training_set), np.array(y_training_set).T[0])
        y_pred = M.predict_proba(np.array(x_test_set))
        y_pred = [x[1] for x in y_pred]
        score.append(error_function(y_test_set,y_pred)) 
        print('.',end=' ')
    average_error = round(sum(score)/folds,4)
    return average_error

In [1]:
def level_1_predictor(t_train,y, model):
    leng = int(len(t_train)/2)
    t_a, t_b= t_train[:leng], t_train[leng:]
    tg_a, tg_b = y[:leng], y[leng:]
    if model == LogisticRegression:
        m1 = model(max_iter = 20**9)
        m2 = model(max_iter = 20**9)
    else:
        m1 = model()
        m2 = model()
    m1.fit(t_a,tg_a)
    tp_b = [i[1] for i in m1.predict_proba(t_b)]
    m2.fit(t_b,tg_b)
    tp_a = [i[1] for i in m2.predict_proba(t_a)]
    
    tp = [[x] for x in np.hstack((tp_a,tp_b))]
    
    return tp

In [2]:
def training_stacker(feature_sets):
    column_name = list(feature_sets.keys())
    itr = 0
    dic_out = {}
    for y in column_name:
        stacking_feature = np.array(feature[feature_sets[y]])
        print('--',column_name[itr], end= ' ')
        score_set = {}
        for x in models:
            l1_pred = level_1_predictor(stacking_feature,np_label.T[0], x)
            score = roc_auc_score(np_label,l1_pred)
            model_name = x.__name__
            score_set[str(score)]=model_name
        model_for_stacking = score_set[str(max([float(x) for x in score_set.keys()]))]
        print(model_for_stacking)
        dic_out[y]=[model_for_stacking]
        itr+=1
    temp = pd.DataFrame(dic_out)
    model_for_stacking_file = pd.read_csv('model_for_stacking_l1.csv')
    model_for_stacking_file = pd.concat((model_for_stacking_file,temp), axis = 0)
    model_for_stacking_file.to_csv('model_for_stacking_l1.csv', index = False)

In [15]:
def update_important_l1(model, file_number):
    i = pd.read_csv('important_l1.csv')
    important_l1 = get_important_feature(model, file_number)
    if model == GradientBoostingClassifier:
        for x in important_l1:
            i.loc[0,x]+=1
    if model == LogisticRegression:
        for x in important_l1:
            i.loc[1,x]+=1
    print(file_number)
    i.to_csv('important_l1.csv', index = False)

In [14]:
def get_important_feature(model,n):
    file = pd.read_csv('important_l1.csv')
    feature_name = file.columns
    if model == GradientBoostingClassifier:
        importance = file.iloc[0]
    if model == LogisticRegression:
        importance = file.iloc[1]
    itr = 0
    important_feature = []
    unimportant_feature = []
    for x in importance.iloc[0]:
        if x>n:
            important_feature.append(feature_name[itr])
        else:
            unimportant_feature.append(feature_name[itr])
        itr+=1
    return important_feature, unimportant_feature

In [7]:
def categorize_important_feature(n):
    important_feature, unimportant_feature = get_important_feature(n)
    delinquency=[]
    spend = []
    payment = []
    balance = []
    risk = []
    std = []
    mean = []
    _25 = []
    _50 = []
    _75 = []
    _min = []
    _max = []

    for x in important_feature:
        if x[0]=='D':
            delinquency.append(x)
        if x[0]=='S':
            spend.append(x)
        if x[0]=='P':
            payment.append(x)
        if x[0]=='B':
            balance.append(x)
        if x[0]=='R':
            risk.append(x)
        if x[-3:]=='std':
            std.append(x)
        elif x[-3:]=='25%':
            _25.append(x)
        elif x[-3:]=='75%':
            _75.append(x)
        elif x[-3:]=='50%':
            _50.append(x)
        elif x[-3:]=='min':
            _min.append(x)
        elif x[-3:]=='max':
            _max.append(x)
        else:
            mean.append(x)  
    feature_sets = {'delinquency':delinquency,'spend':spend,'payment':payment,'balance':balance,'risk':risk,'std':std,
                    'mean':mean,'_25':_25,'_50':_50,'_75':_75,'_min':_min,'_max':_max, 'unimportant_fe':unimportant_feature}
    return feature_sets

In [5]:
def stacker(feature,feature_sets):
    column_name = list(feature_sets.keys())
    r = pd.read_csv('model_for_stacking_l1.csv')   
    stack_feature = pd.DataFrame(columns = column_name)
    for x in column_name:
        print(x, end='-->')
        best_stacking_model = r[x].value_counts().keys()[0]
        stacking_feature = np.array(feature[feature_sets[x]])
        stack_feature[x] = level_1_predictor(stacking_feature,np_label.T[0],model_dic[best_stacking_model])
        print(best_stacking_model)
    return stack_feature

In [6]:
# do run it ... unless you need it
def formating_model_selection():
    model_for_stacking_file = pd.DataFrame(columns = feature_sets.keys())
    model_for_stacking_file.to_csv('model_for_stacking_l1.csv', index = False)
